In [19]:
import torch
import numpy as np
from math import sin, cos

SENTENCE = "I LIKE PIE VERY MUCH PIE ."
DIM_EMB = 512
torch.manual_seed(7)

vocab = dict()
i = 0
for token in SENTENCE.split():
    if token not in vocab:
        vocab[token] = i
        i += 1
vocab["<unk>"] = i
vocab["<s>"] = i + 1
print(vocab)

DICT_SIZE = len(vocab)

oneHotSent = np.zeros((len(SENTENCE.split()), DICT_SIZE), dtype=float)

for i in range(len(SENTENCE.split())):
    word = SENTENCE.split()[i]
    oneHotSent[i][vocab[word]] = 1

oneHotTorch = torch.as_tensor(oneHotSent, dtype=torch.float32)
    
#print(oneHotTorch)
#print(oneHotTorch.shape)
#print(oneHotTorch.dtype)

#DATA PREPARATION DONE

We_skeleton = torch.tensor(np.zeros((DICT_SIZE, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
We = torch.autograd.Variable(We_skeleton, requires_grad=True)
#print(We)
#print(We.shape)
#print(We.dtype)


Batch = oneHotTorch.mm(We)
Pos_emb = np.zeros(Batch.shape, dtype=float)
for p in range(len(SENTENCE.split())):
    for i in range(DIM_EMB//2):
        val = p/(10000**(2*i/DIM_EMB))
        Pos_emb[p][2*i] = sin(val)
        Pos_emb[p][2*i + 1] = cos(val)

Pos_emb_var = torch.autograd.Variable(torch.tensor(Pos_emb, dtype=torch.float32), requires_grad=False)
#print(Pos_emb_var)

#print(Batch)
#Scale batch up
Batch = Batch*(DIM_EMB**(1/2))
#print(Batch)
Batch = Batch + Pos_emb_var
#print(Batch)

#print(Batch.shape)
#print(Batch.dtype)

#Batch (in this case a one sentence batch) is prepared. Maxi calls this embedded sentence

#Create MASK HERE
#
mask = np.triu(np.ones((len(SENTENCE.split()), len(SENTENCE.split()))), k=1)
mask = torch.from_numpy(mask) == 0
print(mask)

# Create self attention
# also need to add a mask here
# @TODO Add a Bias to all Wsz
def add_transformer_head_with_mask(DIM_EMB, Batch, mask):
    Wq_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wq = torch.autograd.Variable(Wq_skeleton, requires_grad=True)

    Wk_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wk = torch.autograd.Variable(Wk_skeleton, requires_grad=True)

    Wv_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wv = torch.autograd.Variable(Wv_skeleton, requires_grad=True)

    Q = Batch.mm(Wq)
    V = Batch.mm(Wv)
    K = Batch.mm(Wk)
    
    tmp1 = Q.mm(torch.t(K))

    tmp = tmp1/(64**(1/2))
    #print(tmp)
    
    softM = torch.nn.Softmax(dim=-1)
    #apply maks here
    tmp = tmp.masked_fill(mask == 0, -1e9)
    
    #print(tmp)

    
    a = softM(tmp)
    print(a)

    z = a.mm(V)
    return z

# @TODO add a bias to Wo
def addLinearProj(concatTransHead, DIM_EMB):
    '''Already concatenataed all heads'''
    Wo_skeleton = torch.tensor(np.zeros((concatTransHead.shape[1], DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wo = torch.autograd.Variable(Wo_skeleton, requires_grad=True)
    
    return concatTransHead.mm(Wo)
    

transformer_head = add_transformer_head_with_mask(DIM_EMB, Batch, mask)
postAttention = addLinearProj(transformer_head, DIM_EMB)
print(transformer_head.shape)
print(postAttention.shape)

# Add residual connection to the embeddings
postAttention = postAttention + Batch

#Normalize

def layerNorm(mat):
    return (mat - mat.mean(dim=0, keepdim=True)) / (mat.std(dim=0, keepdim=True))

normalizedPostAttention = layerNorm(postAttention)

# @TODO no dropout
# Take the output of the attention and do a one layer FF that outputs size of
Wff_skeleton = torch.tensor(np.zeros((DIM_EMB, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
Wff = torch.autograd.Variable(Wff_skeleton, requires_grad=True)
Bff_skeleton = torch.tensor(np.zeros((DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
Bff = torch.autograd.Variable(Bff_skeleton, requires_grad=True)

activation = torch.nn.ReLU()
postFF = activation(normalizedPostAttention.mm(Wff) + Bff)

#Residual
preSoftmaxProj = layerNorm(postFF + normalizedPostAttention)

#Projexct
Wproj_skeleton = torch.tensor(np.zeros((DIM_EMB, DICT_SIZE), dtype=float), dtype=torch.float32,).normal_(0, 1)
Wproj = torch.autograd.Variable(Wproj_skeleton, requires_grad=True)
Bproj_skeleton = torch.tensor(np.zeros((DICT_SIZE), dtype=float), dtype=torch.float32,).normal_(0, 1)
Bproj = torch.autograd.Variable(Bproj_skeleton, requires_grad=True)

linearLay = activation(preSoftmaxProj.mm(Wproj) + Bproj)

#Softmax
softmax = torch.nn.LogSoftmax(dim=1)
output = softmax(linearLay)
print(output.shape)
print(output)



{'I': 0, 'LIKE': 1, 'PIE': 2, 'VERY': 3, 'MUCH': 4, '.': 5, '<unk>': 6, '<s>': 7}
tensor([[1, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1]], dtype=torch.uint8)
tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0.]], grad_fn=<SoftmaxBackward>)
torch.Size([7, 64])
torch.Size([7, 512])
torch.Size([7, 8])
tensor([[-3.7722e+01, -3.7722e+01, -2.8273e+01,  0.0000e+00, -3.7722e+01,
         -1.6410e+01, -3.7722e+01, -3.7722e+01],
        [-3.8353e+01, -3.8353e+01, -2.8882e+01,  0.0000e+00, -3.8353e+01,
         -1.7881e+01, -3.8353e+01, -3.8353e+01],
        [ 0.0000e+00, -2.9192e+01, -2.6355e+01, -3.7078e+01, -3.3044e+01,
       